## Техническое описание кода:

**Функциональность:**

1. **Взаимодействие с LLM:**
    - Класс `BaseAgent` инкапсулирует логику взаимодействия с LLM (конкретно, моделью `gpt-4o-mini` через OpenAI API). 
    - Он принимает системный промпт при инициализации и предоставляет метод `__call__` для отправки сообщений модели и получения ответов.

2. **Оптимизация промпта:**
    - Класс `PromptOptimizer` отвечает за оптимизацию промпта, используя экземпляр `BaseAgent` для связи с LLM. 
    - Он принимает на вход:
        -  исходный промпт, 
        -  данные для обучения (опционально), 
        -  параметры оптимизации (скорость обучения, количество итераций, размер пучка для Монте-Карло, параметры градиентного спуска, критерии оценки в формате JSON).

3. **Генерация кандидатов (`generate_candidates`)**:
    - Создает список потенциально улучшенных промптов на основе текущего, используя два подхода:
        - **Метод Монте-Карло (`generate_monte_carlo_candidate`)**: Генерирует перефразированные версии промпта, запрашивая у LLM "перефразировать, сохраняя смысл".
        - **Градиентный спуск (`generate_gradient_candidate`)**: 
            - Получает ответ LLM на текущий промпт.
            - Вызывает `generate_gradient()` для анализа ответа и получения текстового описания потенциальных улучшений (градиента).
            - Запрашивает у LLM несколько (`steps_per_gradient`) улучшенных вариантов промпта, основываясь на сгенерированном градиенте.

4. **Оценка кандидатов (`evaluate_candidates`)**:
    - Оценивает каждый сгенерированный промпт-кандидат, вызывая `evaluate_response()` и возвращая список оценок.

5. **Оценка ответа (`evaluate_response`)**:
    - **Ключевая функция, определяющая качество ответа LLM на данный промпт.**
    - Анализирует ответ на соответствие критериям, заданным в `evaluation_criteria`:
        - Для каждого критерия вызывает `check_criterion()`, чтобы проверить, выполняется ли условие.
        - Начисляет или снимает баллы в зависимости от важности (`weight`) и обязательности (`required`) критерия.
    - Дополнительно рассчитывает метрики ROUGE и BLEU, сравнивая ответ с исходным промптом, и включает их в общую оценку с заданными весами.
    - Возвращает нормализованную оценку в диапазоне [-1, 1].

6. **Проверка критерия (`check_criterion`)**:
    - Принимает на вход ответ LLM и строку `criterion_check`, содержащую код Python для проверки.
    - Выполняет этот код, передавая `response` как контекст, что позволяет задавать **произвольные критерии оценки** на языке Python.
    - Возвращает True, если условие выполнено, и False в противном случае.

7. **Выбор лучшего кандидата (`select_best_candidate`)**:
    - Использует алгоритм UCB для выбора наилучшего кандидата из списка, балансируя между исследованием новых кандидатов и эксплуатацией уже известных хороших.
    - Учитывает не только среднюю оценку кандидата, но и дисперсию оценок, что позволяет делать более обоснованный выбор в условиях неопределенности.

8. **Генерация градиента (`generate_gradient`)**:
    - **Инновационная функция, автоматизирующая процесс определения критериев оценки и генерации градиента для улучшения промпта.**
    - Отправляет LLM запрос с текущим промптом и просьбой:
        - Определить наиболее важные критерии оценки для этого промпта.
        - Сгенерировать для каждого критерия текстовое описание возможных улучшений (градиент).
    - Ожидает ответ в строго определенном формате JSON, содержащем список критериев с их описанием, важностью, обязательностью и кодом проверки.
    - Анализирует ответ LLM на соответствие сгенерированным критериям.
    - Формирует итоговый текстовый градиент, перечисляя невыполненные критерии и предлагаемые LLM улучшения.

**Механизмы:**

- **Метод Монте-Карло:** Используется для стохастического поиска в пространстве промптов, генерируя случайные вариации.
- **Градиентный спуск:**  Направляет поиск в сторону улучшения промпта, основываясь на анализе ответов LLM и сгенерированных градиентах.
- **Алгоритм UCB:**  Обеспечивает баланс между исследованием новых кандидатов и использованием уже известных хороших, ускоряя сходимость оптимизации.
- **Динамическая генерация критериев оценки:** Делегирует LLM задачу определения критериев оценки, адаптируя процесс оптимизации к конкретным промптам и задачам.
- **Метрики ROUGE и BLEU:** Используются для оценки качества сгенерированного текста путем сравнения его с исходным промптом. ROUGE измеряет степень перекрытия n-грамм между текстами, а BLEU оценивает точность перевода, сравнивая сгенерированный текст с несколькими референтными переводами. 



In [ ]:
# Импорт стандартных библиотек
import logging
import json
import os
import uuid
from pathlib import Path
import rouge
import sacrebleu

import numpy as np
import openai

# Интерфейс для взаимодействия с LLM
from langchain_openai import ChatOpenAI
from langchain_core.messages import BaseMessage, HumanMessage, SystemMessage

from typing import List

In [ ]:
class FileManager:
    """
    Description:
        Класс для управления файлами, включая чтение, запись и добавление содержимого.
    """
    logging.basicConfig(level=logging.INFO)

    def __init__(self, working_directory: str = 'temp'):
        """
        Description:
            Инициализация рабочей директории.

        Args:
            working_directory: Путь к рабочей директории.
        """
        # Создаем рабочую директорию
        self.working_directory = Path(working_directory).absolute()
        logging.info("WORKING_DIRECTORY: %s", self.working_directory)

    @staticmethod
    def generate_run_id() -> str:
        """
        Description:
            Генерирует уникальный UUID.

        Returns:
            str: Сгенерированный UUID.
        """
        return str(uuid.uuid4())

    def read_document(self, file_name: str) -> str:
        """
        Description:
            Читает и возвращает содержимое файла.

        Args:
            file_name (str): Имя файла для чтения.

        Returns:
            str: Содержимое файла.
        """
        return FileManager._read_document(self.working_directory, file_name)

    @staticmethod
    def _read_document(working_directory: Path, file_name: str) -> str:
        """
        Description:
            Вспомогательный метод для чтения содержимого файла.

        Args:
            working_directory: Рабочая директория.
            file_name (str): Имя файла для чтения.

        Returns:
            str: Содержимое файла.
        """
        # Создаем путь к файлу с учетом рабочей директории
        file_path = working_directory / file_name.lstrip('/')
        logging.info(f"Attempting to read file from path: {file_path}")

        try:
            # Открываем файл для чтения
            with file_path.open("r", encoding='utf-8') as file:
                return file.read()
        except FileNotFoundError:
            # Логируем ошибку и возвращаем сообщение об ошибке
            logging.error(f"File {file_name} not found at path: {file_path}")
            return f"File {file_name} not found."

    def write_document(self, content: str, file_name: str) -> str:
        """
        Description:
            Создает и сохраняет текстовый документ.

        Args:
            content: Текстовое содержимое для записи в файл.
            file_name: Имя файла для сохранения.

        Returns:
            str: Сообщение о сохранении файла.
        
        Raises:
            FileNotFoundError: Если файл не найден.
        """
        return FileManager._write_document(self.working_directory, content, file_name)

    @staticmethod
    def _write_document(working_directory: Path, content: str, file_name: str) -> str:
        """
        Description:
            Вспомогательный метод для записи документа.

        Args:
            working_directory: Рабочая директория.
            content: Текстовое содержимое для записи в файл.
            file_name: Имя файла для сохранения.

        Returns:
            str: Сообщение о сохранении файла.
        
        Raises:
            FileNotFoundError: Если файл не найден.
        """
        # Создаем путь к файлу и необходимые директории
        file_path = working_directory / file_name.lstrip('/')
        file_path.parent.mkdir(parents=True, exist_ok=True)
        
        try:
            # Открываем файл для записи
            with file_path.open("w", encoding='utf-8') as file:
                file.write(content)
            return f"Document saved to {file_name}"
        except IOError as e:
            # Логируем ошибку и возвращаем сообщение об ошибке
            logging.error(f"Error writing to file {file_name}: {e}")
            return f"Error writing to file {file_name}: {e}"

    def append_document(self, content: str, file_name: str) -> str:
        """
        Description:
            Добавляет содержимое в конец существующего файла.

        Args:
            content: Текстовое содержимое для добавления в файл.
            file_name: Имя файла для добавления содержимого.

        Returns:
            str: Сообщение о сохранении файла.
        """
        return FileManager._append_document(self.working_directory, content, file_name)

    @staticmethod
    def _append_document(working_directory: Path, content: str, file_name: str) -> str:
        """
        Description:
            Вспомогательный метод для добавления содержимого в документ.

        Args:
            working_directory: Рабочая директория.
            content: Текстовое содержимое для добавления в файл.
            file_name: Имя файла для добавления содержимого.

        Returns:
            str: Сообщение о сохранении файла.
        """
        # Создаем путь к файлу и необходимые директории
        file_path = working_directory / file_name.lstrip('/')
        file_path.parent.mkdir(parents=True, exist_ok=True)
        
        try:
            # Открываем файл для добавления содержимого
            with file_path.open("a", encoding='utf-8') as file:
                file.write(content)
            return f"Document appended to {file_name}"
        except IOError as e:
            # Логируем ошибку и возвращаем сообщение об ошибке
            logging.error(f"Error appending to file {file_name}: {e}")
            return f"Error appending to file {file_name}: {e}"

---
### Test 1

In [ ]:
class BaseAgent:
    """
    Description:
        Класс для взаимодействия с LLM через OpenAI API.

    Attributes:
        system_prompt: Начальный системный промпт для модели.
    """
    def __init__(self, system_prompt: str):
        """
        Description:
            Инициализация класса BaseAgent.

        Args:
            system_prompt: Начальный системный промпт для модели.
        """
        self.system_prompt = system_prompt

    def __call__(self, messages: list) -> str:
        """
        Description:
            Выполнение вызова к LLM с заданными сообщениями.

        Args:
            messages: Список сообщений для отправки в LLM.

        Returns:
            Ответ модели в виде строки.
        """
        response = openai.chat.completions.create(
            model="gpt-4o-mini", 
            messages=[
                {"role": "system", 
                 "content": self.system_prompt},
                *messages
            ]
        ).choices[0].message.content
        return response

---
### Test 2

In [ ]:
# class BaseAgent:
#     """
#     Description:
#         Класс для взаимодействия с моделью ChatOpenAI.
#     """
    
#     def __init__(self, system_prompt: str):
#         """
#         Description:
#             Инициализирует экземпляр CustomChatModel.
            
#         Args:
#             system_prompt: Начальный системный промпт для модели.
#         """
#         self.system_prompt = system_prompt
        
#         self.client = ChatOpenAI(
#             base_url=os.getenv("TGI_URL"),
#             api_key="-",
#             model=os.getenv("MODEL_NAME"),
#             temperature=0.1,
#             n=10,
#             top_p=0.9,
#             max_tokens=4096,
#             streaming=False,
#             verbose=True,
#         )
    
#     def __call__(self, messages: List[BaseMessage]) -> str:
#         """
#         Description:
#             Генерирует ответ от модели на основе предоставленных сообщений.

#         Args:
#             messages (List[BaseMessage]): Список сообщений для отправки модели.

#         Returns:
#             str: Ответ от модели в виде строки.
#         """
#         messages = [
#             SystemMessage(content = self.system_prompt),
#             HumanMessage(content = messages)
#         ]
        
#         # Вызов API
#         response = self.client.invoke(messages)
        
#         return response.content

In [ ]:
class PromptOptimizer:
    """
    Description:
        Класс для оптимизации промпта с использованием методов Монте-Карло и градиентного спуска.

    Attributes:
        llm: Экземпляр класса BaseAgent для взаимодействия с LLM.
        prompt: Исходный промпт для оптимизации.
        num_iterations: Количество итераций оптимизации.
        beam_size: Размер выборки для поиска лучшего кандидата.
        steps_per_gradient: Количество шагов на градиент.
        evaluation_criteria: Конфигурация для оценки ответов модели.
    """
    def __init__(self,
                 llm: BaseAgent,
                 initial_prompt: str,
                 num_iterations: int = 1,
                 beam_size: int = 4,
                 steps_per_gradient: int = 2,
                 evaluation_criteria: str = '{}'):
        """
        Description:
            Инициализация класса PromptOptimizer.

        Args:
            llm: Экземпляр класса BaseAgent для взаимодействия с LLM.
            initial_prompt: Исходный промпт для оптимизации.
            num_iterations: Количество итераций оптимизации.
            beam_size: Размер выборки для поиска лучшего кандидата.
            steps_per_gradient: Количество шагов на градиент.
            evaluation_criteria: Конфигурация для оценки ответов модели в формате JSON.
        """
        self.llm = llm
        self.prompt = initial_prompt
        self.num_iterations = num_iterations
        self.beam_size = beam_size
        self.steps_per_gradient = steps_per_gradient
        self.evaluation_criteria = json.loads(evaluation_criteria)

    def optimize(self) -> str:
        """
        Description:
            Основной метод оптимизации промпта.

        Returns:
            Оптимизированный промпт.
        """
        for i in range(self.num_iterations):
            print(f"[INFO] === Iteration {i+1}/{self.num_iterations} ===")
            # Генерируем кандидатов на основе текущего промпта
            candidates = self.generate_candidates()
            # Оцениваем каждого кандидата
            candidate_scores = self.evaluate_candidates(candidates, self.evaluation_criteria)
            # Выбираем лучшего кандидата по результатам оценки
            best_candidate_idx = self.select_best_candidate(candidate_scores)
            # Обновляем текущий промпт лучшим кандидатом
            self.prompt = candidates[best_candidate_idx]
            print(f"[INFO] Best prompt at iteration {i+1}: {self.prompt}\n")
        return self.prompt

    def generate_candidates(self) -> list:
        """
        Description:
            Генерация списка кандидатов на основе исходного промпта.

        Returns:
            Список сгенерированных кандидатов.
        """
        candidates = [self.prompt]
        
        # Генерация кандидатов с использованием метода Монте-Карло
        for _ in range(self.beam_size):
            monte_carlo_candidate = self.generate_monte_carlo_candidate(self.prompt)
            candidates.append(monte_carlo_candidate)
        
        # Генерация кандидатов на основе градиентного спуска
        for _ in range(self.beam_size):
            gradient_candidate = self.generate_gradient_candidate(self.prompt)
            candidates.append(gradient_candidate)
        
        return candidates

    def generate_monte_carlo_candidate(self, prompt: str) -> str:
        """
        Description:
            Генерация кандидата с использованием метода Монте-Карло.

        Args:
            prompt: Исходный промпт.

        Returns:
            Сгенерированный промпт.
        """
        paraphrased_prompt = self.llm(messages=[{"role": "user", "content": f"Перефразируй следующий текст, сохраняя смысл:\n{prompt}"}])
        return paraphrased_prompt

    def generate_gradient_candidate(self, prompt: str) -> str:
        """
        Description:
            Генерация кандидата с использованием градиентного подхода.

        Args:
            prompt: Исходный промпт.

        Returns:
            Сгенерированный промпт с улучшениями.
        """
        # Получаем ответ от LLM на исходный промпт
        response = self.llm(messages=[{"role": "user", "content": prompt}])

        # Генерируем текстовые улучшения на основе анализа ответа LLM
        gradient = self.generate_gradient(prompt, response)

        # Запрашиваем у LLM улучшенные версии промпта на основе предложенных улучшений
        improved_prompt = self.llm(messages=[{"role": "user", "content": f"""
            Я пытаюсь написать промпт для ИИ-помощника.
            Мой текущий промпт:
            ```
            {prompt}
            ```
            Но он не идеален: {gradient}
            Учитывая вышеизложенное, я написал {self.steps_per_gradient} различных улучшенных промпта.
            Каждый промпт заключен в ```:
            """}])
        
        return improved_prompt

    def evaluate_candidates(self, candidates: list, evaluation_criteria: dict) -> list:
        """
        Description:
            Оценка кандидатов на основе заданных критериев.

        Args:
            candidates: Список кандидатов.
            evaluation_criteria: Словарь с критериями оценки.

        Returns:
            Список оценок для каждого кандидата.
        """
        print("[INFO] Evaluating candidates...")
        
        candidate_scores = []

        for candidate in candidates:
            # Получаем ответ от LLM на каждый кандидатский промпт
            response = self.llm(messages=[{"role": "user", "content": candidate}])
            # Оцениваем ответ на основе заданных критериев
            score = self.evaluate_response(response, evaluation_criteria)
            candidate_scores.append(score)
        
        print(f"[INFO] Candidate scores: {candidate_scores}")
        return candidate_scores

    def evaluate_response(self, response: str, evaluation_criteria: dict) -> float:
        """
        Description:
            Оценивает ответ LLM на основе заданных критериев, переданных в конфигурации.

        Args:
            response: Ответ модели.

        Returns:
            Оценка (число), представляющая качество ответа.
        """
        score = 0
        total_possible_score = 0
        
        for criterion in evaluation_criteria.get("criteria", []):
            criterion_name = criterion.get("name")
            criterion_required = criterion.get("required", False)
            criterion_check = criterion.get("check", "True")
            criterion_weight = criterion.get("weight", 1)
            total_possible_score += criterion_weight

            try:
                # Проверяем выполнение условия для критерия
                criterion_met = self.check_criterion(response, criterion_check)
                print(f"[DEBUG] Evaluating criterion '{criterion_name}': Met = {criterion_met}, Required = {criterion_required}, Weight = {criterion_weight}")
                
                if criterion_met:
                    score += criterion_weight
                elif criterion_required:
                    # Если критерий обязателен и не выполнен, снижаем итоговый балл
                    score -= criterion_weight
                else:
                    print(f"[WARN] Criterion '{criterion_name}' not met, but not required.")
            except Exception as e:
                print(f"[ERROR] Error evaluating criterion '{criterion_name}': {e}")
        
        # Вычисление метрик ROUGE и BLEU
        rouge_scorer = rouge.Rouge()
        rouge_scores = rouge_scorer.get_scores(response, self.prompt)[0]
        print(f"[INFO] ROUGE scores: {rouge_scores}")

        bleu_score = sacrebleu.corpus_bleu([response], [[self.prompt]]).score
        print(f"[INFO] BLEU score: {bleu_score}")

        # Включение ROUGE и BLEU в итоговую оценку
        rouge_weight = evaluation_criteria.get("rouge_weight", 0.2)
        bleu_weight = evaluation_criteria.get("bleu_weight", 0.2)

        total_possible_score += rouge_weight + bleu_weight
        score += rouge_scores['rouge-l']['f'] * rouge_weight
        score += bleu_score / 100 * bleu_weight

        # Нормализуем оценку в диапазоне от -1 до 1
        normalized_score = score / total_possible_score if total_possible_score > 0 else 0
        print(f"[INFO] Total score for the response: {score} / {total_possible_score} (Normalized: {normalized_score})")
        print('=' * 101)

        return max(min(normalized_score, 1), -1)  # Ограничиваем результат в диапазоне [-1, 1]


    def check_criterion(self, response: str, criterion_check: str) -> bool:
        """
        Description:
            Проверка ответа LLM на соответствие критерию.

        Args:
            response: Ответ LLM.
            criterion_check: Строка с условием проверки.

        Returns:
            True, если ответ соответствует критерию, иначе False.
        """
        # Формируем запрос к LLM, чтобы она проверила соответствие ответа критерию
        prompt = f"""
        Вход: 
        Ответ: \"{response}\"
        Критерий: \"{criterion_check}\"

        Вопрос: Выполняется ли критерий для данного ответа? Ответь 'True', если да, и 'False', если нет. 
        """
        
        # Получаем ответ от LLM на запрос
        llm_response = self.llm(messages=[{"role": "user", "content": prompt}])
        
        # Обрабатываем ответ модели
        llm_response = llm_response.strip().lower()
        
        # Проверяем, является ли ответ True или False
        if llm_response in ["true", "false"]:
            return llm_response == "true"
        else:
            print(f"[ERROR] Unexpected LLM response for criterion check: {llm_response}")
            return False

    def select_best_candidate(self, candidate_scores: list) -> int:
        """
        Description:
            Реализация алгоритма UCB для выбора лучшего кандидата.

        Args:
            candidate_scores: Оценки кандидатов.

        Returns:
            Индекс лучшего кандидата.
        """
        print("[INFO] Selecting the best candidate using UCB...")

        N = len(candidate_scores)       # Общее количество кандидатов
        T = np.arange(1, N + 1)         # Массив для отслеживания количества испытаний каждого кандидата
        Q = np.array(candidate_scores)  # Оценки кандидатов

        # Вычисление дисперсии оценок кандидатов
        variance = np.var(Q)
        print(f"[DEBUG] Variance of candidate scores: {variance}")

        # Расчет UCB с учетом дисперсии
        ucb_values = Q + np.sqrt(2 * np.log(T) / (T + 1e-5)) + variance

        print(f"[DEBUG] UCB values: {ucb_values}")

        # Выбор индекса кандидата с максимальным значением UCB
        best_idx = np.argmax(ucb_values)
        
        print(f"[INFO] Selected best candidate index: {best_idx}")
        
        return best_idx  # Возвращаем индекс лучшего кандидата

    def generate_gradient(self, prompt: str, response: str) -> str:
        """
        Description:
            Генерация градиента на основе динамического анализа ответа LLM и критериев оценки, заданных для конкретного промпта.

        Args:
            prompt: Исходный промпт.
            response: Ответ LLM на этот промпт.

        Returns:
            Текстовый градиент, указывающий на возможные улучшения.
        """
        # Шаг 1: Запрашиваем у LLM критерии оценки и градиенты для данного промпта
        evaluation_criteria_json = self.llm(messages=[{
            "role": "user",
            "content": f"""
                Мне нужно оценить ответ модели на следующий промпт:
                ```
                {prompt}
                ```
                В зависимости от этого промпта, определи какие критерии оценки наиболее важны для ответа модели. 
                Учти, что критерии должны быть основаны на содержании промпта, его цели, ожидаемом результате и других релевантных факторах. 

                Определи критерии оценки этого промпта и представь их в формате JSON.
                Также сформируй список возможных улучшений (градиентов) для каждого критерия в формате JSON.
                Пример ответа:
                {{
                    "criteria": [
                        {{
                            "name": "include_math",
                            "required": true,
                            "check": "response.count('math') > 0",
                            "gradient": "Добавьте математические формулы для лучшего объяснения.",
                            "weight": 0.3
                        }},
                        {{
                            "name": "latex_formatting",
                            "required": true,
                            "check": "response.count('\\$') > 0",
                            "gradient": "Убедитесь, что формулы правильно отформатированы в LaTeX.",
                            "weight": 0.3
                        }},
                        {{
                            "name": "clarity_and_conciseness",
                            "required": true,
                            "check": "len(response.split()) < 100",
                            "gradient": "Обеспечьте ясность и краткость в объяснениях.",
                            "weight": 0.2
                        }},
                        {{
                            "name": "detailed_explanation",
                            "required": true,
                            "check": "response.count('.') > 2",
                            "gradient": "Расширьте объяснение для повышения его полноты.",
                            "weight": 0.2
                        }}
                    ]
                }}
            Правила ответа:
            - ВСЕГДА используйте пример ответа для структуры сообщения.
            - ОТВЕЧАЙ ТОЛЬКО В ФОРМАТЕ JSON, КАК УКАЗАНО В ПРИМЕРЕ ОТВЕТА.
            - НЕ ДОБАВЛЯЙ ЛЮБЫЕ ДОПОЛНИТЕЛЬНЫЕ КОММЕНТАРИИ ИЛИ ВСТУПИТЕЛЬНЫЕ ФРАЗЫ.
            - СТРОГО следуйте примеру ответа!
            """
        }])
        # Проверяем, что ответ не пустой
        if not evaluation_criteria_json:
            raise ValueError("Получен пустой ответ от LLM")

        # Парсим JSON с критериями и градиентами
        try:
            self.evaluation_criteria = json.loads(evaluation_criteria_json)
        except json.JSONDecodeError as e:
            return ""

        # Шаг 2: Анализируем ответ LLM на соответствие критериям и формируем градиенты
        gradients = []
        
        for criterion in self.evaluation_criteria.get("criteria", []):
            criterion_name = criterion.get("name")
            criterion_required = criterion.get("required")
            criterion_gradient = criterion.get("gradient")
            criterion_check = criterion.get("check")
            

            if criterion_required and not self.check_criterion(response, criterion_check):
                print(f"[INFO] Criterion '{criterion_name}' not met, adding gradient: {criterion_gradient}")
                gradients.append(criterion_gradient)

        # Шаг 3: Возвращаем сгенерированные градиенты
        generated_gradient = " ".join(gradients)
        
        return generated_gradient


In [ ]:
# Пример использования
initial_prompt = """
You are a top-level supervisor tasked with managing a conversation between the following teams: {team_members}.
Given the following user request, respond with the team to act next.
Each team will perform a task and respond with their results and status.

INSTRUCTIONS:
If InformationCollectionTeam responded with an error, this means the user submitted incorrect data. Finish the job, select FINISH.
If a microservice does not have API methods, then it will not be possible to generate documentation. Finish the job, select FINISH.

When finished, respond with FINISH.
"""

# Инициализация файл-менеджера
file_manager = FileManager()

llm = BaseAgent(system_prompt = file_manager.read_document('prompts/prompt_engineering.txt'))

optimizer = PromptOptimizer(llm, initial_prompt)
optimized_prompt = optimizer.optimize()

print("Оптимизированный промпт:", optimized_prompt)

## Интерпретация результатов и метрик

#### 1. Критерий "testing_and_validation"
- **Met = False:** Критерий не был выполнен, что означает, что ответ не соответствует необходимым требованиям по тестированию и валидации.
- **Required = True:** Этот критерий является обязательным, поэтому его невыполнение снижает общую оценку.
- **Weight = 0.25:** Критерий имеет вес 0.25, что означает, что его невыполнение оказывает значительное влияние на итоговую оценку.

#### 2. Метрики ROUGE
- **ROUGE-1 (r = 0.4, p = 0.141, f = 0.209):** Эта метрика оценивает схожесть на уровне униграмм (отдельных слов) между сгенерированным ответом и эталонным текстом. Значение f-score 0.209 указывает на то, что схожесть относительно низкая, особенно в контексте precision (точности), что означает, что многие слова в ответе не соответствуют эталонным.
- **ROUGE-2 (r = 0.142, p = 0.045, f = 0.068):** Эта метрика оценивает схожесть на уровне биграмм (пар слов). Значение f-score 0.068 свидетельствует о ещё меньшей схожести на уровне пар слов, что указывает на проблемы с построением связных фраз.
- **ROUGE-L (r = 0.4, p = 0.141, f = 0.209):** Эта метрика оценивает схожесть на уровне длинных совпадений последовательностей (например, фраз). Значение f-score 0.209 подтверждает низкую схожесть с эталонным текстом.

#### 3. BLEU Score (4.99)
- BLEU (Bilingual Evaluation Understudy) оценивает точность и полноту перевода или текста по отношению к эталонному. Значение 4.99 указывает на очень низкую точность и схожесть с эталоном. В идеале, BLEU score должен быть ближе к 100, что указывает на высокий уровень схожести с эталоном.

#### 4. Общий результат
- **Total score for the response: -0.948 / 1.4 (Normalized: -0.677):** Общая оценка показывает, что ответ получил отрицательную оценку из-за невыполнения обязательного критерия и низких результатов по метрикам ROUGE и BLEU. Нормализованное значение -0.677 указывает на то, что ответ значительно отклонился от ожидаемого качества.
- **Candidate scores:** Все кандидаты получили отрицательные оценки, что свидетельствует о том, что ни один из предложенных промптов не показал удовлетворительного результата.

#### 5. Алгоритм UCB
- **UCB values:** Алгоритм UCB использовался для выбора наилучшего кандидата на основе оценок и дисперсии. Значения UCB показывают, что несмотря на низкие оценки всех кандидатов, один из них имеет потенциально лучший баланс между оценкой и дисперсией (в данном случае, кандидат под индексом 2).
- **Selected best candidate index: 2:** Несмотря на то, что все кандидаты показали низкие результаты, алгоритм выбрал кандидата под индексом 2 как наилучший вариант на основе UCB значений.
